In [1]:
import pandas as pd

from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression


from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn import metrics

from sklearn.ensemble import VotingClassifier

import warnings
warnings.filterwarnings('ignore')

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/titaniccleaningfeatureengineering/__results__.html
/kaggle/input/titaniccleaningfeatureengineering/testC.csv
/kaggle/input/titaniccleaningfeatureengineering/__notebook__.ipynb
/kaggle/input/titaniccleaningfeatureengineering/custom.css
/kaggle/input/titaniccleaningfeatureengineering/trainC.csv
/kaggle/input/titaniccleaningfeatureengineering/__output__.json
/kaggle/input/titaniccleaningfeatureengineering/__results___files/__results___13_0.png
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/train.csv
/kaggle/input/titanic/gender_submission.csv


In [2]:
train = pd.read_csv('../input/titaniccleaningfeatureengineering/trainC.csv')
test = pd.read_csv('../input/titaniccleaningfeatureengineering/testC.csv')
submission = pd.read_csv('../input/titanic/gender_submission.csv')

In [3]:
sc = MinMaxScaler()
X = sc.fit_transform(train[train.columns[1:]])
scaled_test = sc.fit_transform(test)
y = train['Survived']

In [4]:
clf = VotingClassifier(estimators = 
                                    [('KNN', KNeighborsClassifier()),
                                    ('RFC', RandomForestClassifier()),
                                    ('LogReg', LogisticRegression()),
                                    ('GBRT', GradientBoostingClassifier())], 
                                    voting = 'soft')

params = [{'KNN__n_neighbors' : [4,5,6], 'KNN__weights' : ['uniform','distance'], 'KNN__p' : [2,3,4],
           'RFC__n_estimators' : [100],
           'LogReg__C': [1, 10, 100], 'LogReg__penalty' : ['l1', 'l2'],
          }]

grid = GridSearchCV(clf, params, cv=5, scoring='accuracy', n_jobs=-1)
grid.fit(X,y)

GridSearchCV(cv=5, error_score=nan,
             estimator=VotingClassifier(estimators=[('KNN',
                                                     KNeighborsClassifier(algorithm='auto',
                                                                          leaf_size=30,
                                                                          metric='minkowski',
                                                                          metric_params=None,
                                                                          n_jobs=None,
                                                                          n_neighbors=5,
                                                                          p=2,
                                                                          weights='uniform')),
                                                    ('RFC',
                                                     RandomForestClassifier(bootstrap=True,
                                  

In [5]:
print (grid.best_score_)
print (grid.best_params_)
print(grid.best_estimator_)

0.9326784257108782
{'KNN__n_neighbors': 4, 'KNN__p': 2, 'KNN__weights': 'distance', 'LogReg__C': 100, 'LogReg__penalty': 'l2', 'RFC__n_estimators': 100}
VotingClassifier(estimators=[('KNN',
                              KNeighborsClassifier(algorithm='auto',
                                                   leaf_size=30,
                                                   metric='minkowski',
                                                   metric_params=None,
                                                   n_jobs=None, n_neighbors=4,
                                                   p=2, weights='distance')),
                             ('RFC',
                              RandomForestClassifier(bootstrap=True,
                                                     ccp_alpha=0.0,
                                                     class_weight=None,
                                                     criterion='gini',
                                                     max_dep

In [6]:
clf_grid = grid.best_estimator_
clf_grid.score(X,y)

0.9966329966329966

In [7]:
cross = cross_val_score(clf_grid, X, y, cv = 10, scoring = "accuracy", n_jobs=-1)
print('The cross validated score is', cross.mean())

The cross validated score is 0.9326841448189764


In [8]:
submission['Survived'] = clf_grid.predict(scaled_test)
submission.to_csv("submission_votingclassifier.csv", index = False)
submission.head()

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1
